In [ ]:
!ls

In [172]:
import numpy as np
import scipy.optimize as spo

In [5]:
def sigmoid(x):
    """ Evaluate the sigmoid function on an np.array
    
        Keyword Arguments:
        x -- an np.array.
    """
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_gradient(x):
    """ Evaluate the sigmoid gradient (i.e. the derivative of the sigmoid function)
        
        Keyword Arguments:
        x -- an np.array
    """
    q = sigmoid(x)
    return q * (1 - q)

In [46]:
def cost_function(nn_params, il_size, hl_size, num_labels, X, y, lam):
    """ Evaluate the cost function for a layer of a backpropogated neural net.
    
        Keyword Arguments:
        
        nn_params -- A rolled-up np.array containing the theta parameters of the net.
        il_size -- The input layer size.
        hl_size -- The hidden layer size.
        num_labels -- The number of output neurons, i.e. the output layer size
        X -- Feature matrix, with training examples in rows
        y -- Label vector, corresponding to output neurons 1 ... num_labels
        lam -- regularization parameter
    """
    split_indx = hl_size * (il_size + 1);
    
#     print(split_indx)
#     print(len(nn_params[0:split_indx]))

    Theta1 = np.reshape(nn_params[0:split_indx], [hl_size, il_size + 1])
    Theta2 = np.reshape(nn_params[split_indx:] , [num_labels, hl_size + 1] )
    
#     print(Theta1.shape)
#     print(Theta2.shape)

    m = X.shape[0]

    J = 0
    
    K = num_labels
    for i in range(0,m):
        # Forward propogation
        a_1 = np.insert(X[i,:], 0, 1)
        
        z_2 = np.dot(Theta1, a_1)
        a_2 = np.insert(sigmoid(z_2), 0, 1)
        
        z_3 = np.dot(Theta2, a_2)
        a_3 = sigmoid(z_3)
        
        yvec = np.zeros([num_labels,1])
        yvec[y[i]-1] = 1;
        
        term1 = np.dot( np.transpose(-1 * yvec), np.log(a_3))
        term2 = np.dot( np.transpose(1 - yvec) , np.log(1 - a_3) )

        J = J + np.sum( term1 - term2);
        
        Theta1_temp = Theta1[:,1:]
        Theta2_temp = Theta2[:,1:]
        regterm = (lam / (2 * m)) * (np.sum(np.power(Theta1_temp.flatten(),2)) + np.sum(np.power(Theta2_temp.flatten(),2)))
        J = J + regterm
    return J/m

In [154]:
def jacobian(nn_params, il_size, hl_size, num_labels, X, y, lam):
    """ Evaluate the Jacobian (i.e. gradient) for a layer of a backpropogated neural net.
    
        Keyword Arguments:
        
        nn_params -- A rolled-up np.array containing the theta parameters of the net.
        il_size -- The input layer size.
        hl_size -- The hidden layer size.
        num_labels -- The number of output neurons, i.e. the output layer size
        X -- Feature matrix, with training examples in rows
        y -- Label vector, corresponding to output neurons 1 ... num_labels
        lam -- regularization parameter
    """
    split_indx = hl_size * (il_size + 1);
    
    m = X.shape[0]
    
    Theta1 = np.reshape(nn_params[0:split_indx], [hl_size, il_size + 1])
    Theta2 = np.reshape(nn_params[split_indx:] , [num_labels, hl_size + 1] )
    
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    Delta_1 = np.zeros(Theta1.shape)
    Delta_2 = np.zeros(Theta2.shape)
    for i in range(0,m):
        # Forward propogation
        a_1 = np.insert(X[i,:], 0, 1)
        a_1.shape = (il_size + 1, 1)
        
        z_2 = np.dot(Theta1, a_1)
        a_2 = np.insert(sigmoid(z_2), 0, 1)
        a_2.shape = (hl_size + 1, 1)
        
        z_3 = np.dot(Theta2, a_2)
        a_3 = sigmoid(z_3)
        a_3.shape = (num_labels,1)
        yvec = np.zeros([num_labels,1])
        yvec[y[i]-1] = 1;
                
        d_3 = a_3 - yvec
        siggrad = sigmoid_gradient(z_2).flatten()
        siggrad.shape = (hl_size,1)

        d_2 = np.multiply(np.dot(np.transpose(Theta2[:,1:]), d_3), siggrad)
        
#         print(Delta_2.shape)
#         print(np.dot(d_3, np.transpose(a_2)).shape)
#         print(Delta_1.shape)
#         print(d_2.shape)
#         print(np.transpose(a_1).shape)
#         print(np.dot(d_2, np.transpose(a_1)).shape)
        Delta_2 = Delta_2 + np.dot(d_3, np.transpose(a_2))
        Delta_1 = Delta_1 + np.dot(d_2, np.transpose(a_1))
    
    Theta1_grad = Delta_1 * 1/m;
    Theta2_grad = Delta_2 * 1/m;
    
    Theta1_grad[:,1:] = Theta1_grad[:,1:] + lam/m * Theta1[:,1:]
    Theta2_grad[:,1:] = Theta2_grad[:,1:] + lam/m * Theta2[:,1:]
    t1 = Theta1_grad.flatten()
    t2 = Theta2_grad.flatten()
    
    return np.concatenate([t1, t2])

In [184]:
def rand_init_weights(lin, lout):
    epsilon_init = .1;
    W = np.random.randn(lout, 1 + lin) * epsilon_init;
    return W

In [185]:
# "Live" Section
INPUT_LAYERS = 4
HIDDEN_LAYERS = 10
OUTPUT_LAYERS = 3

init_theta1 = rand_init_weights(INPUT_LAYERS, HIDDEN_LAYERS).flatten()
init_theta2 = rand_init_weights(HIDDEN_LAYERS, OUTPUT_LAYERS).flatten()

init_params = np.concatenate([init_theta1, init_theta2])

X = np.array([[1,2,3,4],[4,5,6,7],[7,8,9,10]])
y = np.array([1,2,3])

lam = .1

params = (INPUT_LAYERS, HIDDEN_LAYERS, OUTPUT_LAYERS, X, y, lam)

spo.minimize(cost_function, init_params, method = 'BFGS', args = params, jac = jacobian, options = {'disp':True})

Optimization terminated successfully.
         Current function value: 0.738642
         Iterations: 205
         Function evaluations: 208
         Gradient evaluations: 208


      fun: 0.7386415237086855
 hess_inv: array([[  3.28047587e+02,   4.98247969e-01,  -6.28900119e+00, ...,
          9.41775467e+00,   4.99897768e-01,   1.58559389e+00],
       [  4.98247969e-01,   5.15751029e+00,   1.49671553e+00, ...,
         -4.06313271e-01,   1.58911642e+00,  -5.04025540e-01],
       [ -6.28900119e+00,   1.49671553e+00,   1.61925583e+00, ...,
         -4.16014804e-01,   6.75179249e-01,  -2.62641441e-01],
       ..., 
       [  9.41775467e+00,  -4.06313271e-01,  -4.16014804e-01, ...,
          1.17199880e+01,   3.84028007e+00,   2.51783269e+00],
       [  4.99897768e-01,   1.58911642e+00,   6.75179249e-01, ...,
          3.84028007e+00,   1.28084895e+01,   2.09452116e+00],
       [  1.58559389e+00,  -5.04025540e-01,  -2.62641441e-01, ...,
          2.51783269e+00,   2.09452116e+00,   5.61637649e+00]])
      jac: array([  6.90878205e-07,  -1.31753339e-06,  -3.45610063e-07,
        -9.50830214e-08,   1.01929575e-06,   8.78634575e-07,
        -1.78162358e-06,   6.954

In [159]:
ws = 10*(4 + 1) + 11*3
tparams = np.ones(ws)
j = jacobian(tparams, 4, 10, 3, np.array([[1,2,3,4],[4,5,6,7],[7,8,9,10]]), np.array([1,2,3]), 4)
print(np.mean(j))
print(np.sum(j))

1.38956500663
115.333895551


In [47]:
ws = 10*(4 + 1) + 11*3
tparams = np.ones(ws)
cost_function(tparams, 4, 10, 3, np.array([[1,2,3,4],[4,5,6,7],[7,8,9,10]]), np.array([1,2,3]), 4)

68.666605430649767

In [167]:
rand_init_weights(3,5)

array([[-0.02398327,  0.00288636, -0.02521148, -0.05847296,  0.09419758],
       [-0.10979766, -0.1853879 , -0.00166844, -0.06133563,  0.10034813],
       [-0.11266522, -0.10412894,  0.11466746,  0.18751797, -0.11715688]])

In [56]:
?np.dot

In [132]:
a = np.array([[1,2,3],[4,5,6]]) # 2x3
b = np.array([[1,2,3,4], [5,6,7,8], [1,2,3,5]]) #3x4 
print(a)
np.multiply(a,b)

[[1 2 3]
 [4 5 6]]


ValueError: operands could not be broadcast together with shapes (2,3) (3,4) 

In [62]:
?np.reshape